<a href="https://colab.research.google.com/github/grayzerguy/02-homework-solution/blob/master/ZARA_XML_TO_EXCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import xml.etree.ElementTree as ET
import pandas as pd

# שלב 1: העלאת הקובץ
uploaded = files.upload()
filename = next(iter(uploaded))  # לוקח את שם הקובץ שהועלה

# שלב 2: עיבוד הקובץ
tree = ET.parse(filename)
root = tree.getroot()
ns = {'ns': 'http://www.inditex.com/Schemas/Ecommerce/ItxShipmentRequest'}

rows = []

for order in root.findall('.//ns:Order', ns):
    order_data = order.attrib
    for parcel in order.find('ns:Parcels', ns).findall('ns:Parcel', ns):
        parcel_data = parcel.attrib
        row = {
            "OrderNumber": order_data.get("OrderNumber", ""),
            "CustomerName": order_data.get("CustomerName", ""),
            "CustomerSurname": order_data.get("CustomerSurname", ""),
            "Phone": order_data.get("PhoneNumber1", ""),
            "City": order_data.get("City", ""),
            "AddressLine1": order_data.get("AddressLine1", ""),
            "AddressLine2": order_data.get("AddressLine2", ""),
            "ParcelRef": parcel_data.get("InditexBoxReference", ""),
            "CourierRef": parcel_data.get("CourierBoxReference", ""),
            "Weight": parcel_data.get("Weight", ""),
            "PackageType": parcel_data.get("PackageType", "")
        }
        rows.append(row)

# יצירת קובץ Excel
df = pd.DataFrame(rows)
output_file = "parcels_output.xlsx"
df.to_excel(output_file, index=False)

# שלב 3: הורדה
files.download(output_file)
